Kaggle link: https://www.kaggle.com/duygus/gretta-gretta?scriptVersionId=34868497

Environment setup

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bq_helper import BigQueryHelper
from google.cloud import bigquery
# Set use_legacy_sql to True to use legacy SQL syntax.
job_config = bigquery.QueryJobConfig()
job_config.use_legacy_sql = True

Start with getting the big data with BigQueryHelper

In [2]:
noaa_dataset = BigQueryHelper(
        active_project= "bigquery-public-data", 
        dataset_name = "noaa_gsod")
noaa_dataset.list_tables()

Using Kaggle's public dataset BigQuery integration.


['gsod1929',
 'gsod1930',
 'gsod1931',
 'gsod1932',
 'gsod1933',
 'gsod1934',
 'gsod1935',
 'gsod1936',
 'gsod1937',
 'gsod1938',
 'gsod1939',
 'gsod1940',
 'gsod1941',
 'gsod1942',
 'gsod1943',
 'gsod1944',
 'gsod1945',
 'gsod1946',
 'gsod1947',
 'gsod1948',
 'gsod1949',
 'gsod1950',
 'gsod1951',
 'gsod1952',
 'gsod1953',
 'gsod1954',
 'gsod1955',
 'gsod1956',
 'gsod1957',
 'gsod1958',
 'gsod1959',
 'gsod1960',
 'gsod1961',
 'gsod1962',
 'gsod1963',
 'gsod1964',
 'gsod1965',
 'gsod1966',
 'gsod1967',
 'gsod1968',
 'gsod1969',
 'gsod1970',
 'gsod1971',
 'gsod1972',
 'gsod1973',
 'gsod1974',
 'gsod1975',
 'gsod1976',
 'gsod1977',
 'gsod1978',
 'gsod1979',
 'gsod1980',
 'gsod1981',
 'gsod1982',
 'gsod1983',
 'gsod1984',
 'gsod1985',
 'gsod1986',
 'gsod1987',
 'gsod1988',
 'gsod1989',
 'gsod1990',
 'gsod1991',
 'gsod1992',
 'gsod1993',
 'gsod1994',
 'gsod1995',
 'gsod1996',
 'gsod1997',
 'gsod1998',
 'gsod1999',
 'gsod2000',
 'gsod2001',
 'gsod2002',
 'gsod2003',
 'gsod2004',
 'gsod2005',

In [3]:
# Let's see what's inside
noaa_dataset.head('stations', num_rows=10)

,usaf,wban,name,country,state,call,lat,lon,elev,begin,end
0,007018,99999,WXPOD 7018,None,None,None,0.000,0.000,+7018.0,20110309,20130730
1,007026,99999,WXPOD 7026,AF,None,None,0.000,0.000,+7026.0,20120713,20170822
2,007070,99999,WXPOD 7070,AF,None,None,0.000,0.000,+7070.0,20140923,20150926
3,008268,99999,WXPOD8278,AF,None,None,32.950,65.567,+1156.7,20100519,20120323
4,008307,99999,WXPOD 8318,AF,None,None,0.000,0.000,+8318.0,20100421,20100421
5,010016,99999,RORVIK/RYUM,NO,None,None,64.850,11.233,+0014.0,19870116,19910806
6,010017,99999,FRIGG,NO,None,ENFR,59.980,2.250,+0048.0,19880320,20050228
7,010071,99999,LONGYEARBYEN,SV,None,None,78.217,15.583,+0037.0,20050210,20050210
8,010190,99999,SVARTTANGEN,NO,None,None,77.517,20.817,+0020.0,20100825,20140523
9,010303,99999,TROMSO/SKATTURA,NO,None,None,69.700,19.017,+0014.0,20140522,20150108


Let's get the data from NJ, using SQL.

In [4]:
query = """
    SELECT *
    FROM `bigquery-public-data.noaa_gsod.stations`
    WHERE state = "NJ"
"""
NJ_stations = noaa_dataset.query_to_pandas(query)
NJ_stations[NJ_stations['name'].str.contains('NEWARK')]

,usaf,wban,name,country,state,call,lat,lon,elev,begin,end
26,999999,14734,NEWARK INTERNATIONAL ARPT,US,NJ,KEWR,40.683,-74.169,+0009.1,19480101,19721231
56,725020,14734,NEWARK LIBERTY INTERNATIONAL,US,NJ,KEWR,40.683,-74.169,+0002.1,19730101,20200516


Get all the weather data coming from Newark Intl Airport Station, with the stn number '725020'

In [5]:
# Get all the weather data coming from Newark Intl Airport Station
query = """SELECT year,mo,da,temp,dewp,visib,wdsp,prcp,fog,rain_drizzle,snow_ice_pellets,hail,thunder,tornado_funnel_cloud
    FROM `bigquery-public-data.noaa_gsod.gsod*`
    WHERE stn = '725020'
    """
# Estimate the size
noaa_dataset.estimate_query_size(query)

11.236002331599593

Sort and write the data into a csv 

In [6]:
# take Newark data 
newark_data = noaa_dataset.query_to_pandas(query)
# newark_data.to_csv('noaa_newark_data.csv')
newark_data_sorted = newark_data.sort_values(by=['year','mo','da']).reset_index().drop('index',axis=1)
newark_data_sorted.to_csv('newark_data_sorted.csv', index=False)

Get more data from other states: this time NJ, NY, LA, and WI

The stations we picked:
*     NJ: 725020 Newark Intl Airport Station,
*     NY: 725030 NYC/LA GUARDIA,
*     CA: 722950 LOS ANGELES,
*     WI: 727410 PARK FALLS

This time sort the data with SQL

In [7]:
query = """
    SELECT *
    FROM `bigquery-public-data.noaa_gsod.gsod*`
    WHERE stn IN ('725020','725030','722950','727410')
    ORDER BY year, mo, da
    """
noaa_dataset.estimate_query_size(query)

27.21099486760795

In [8]:
all_states = noaa_dataset.query_to_pandas(query)

In [9]:
all_states.to_csv('all_states_data_sorted.csv', index=False)